# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

In [2]:
import re

from pprint import pprint
from typing import Callable

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [3]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)

    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте

    for text in train_text:
        tokens = re.split(r"\s+", text.strip())

        if bos:
            tokens = ["<s>"] + tokens

        if eos:
            tokens = tokens + ["</s>"]

        for n in range(0, order):
            for pos in range(0, len(tokens)):
                ngram = tuple(tokens[pos:pos+n+1])

                if len(ngram) < n + 1:
                    continue

                if ngram not in ngrams:
                    ngrams[ngram] = 0

                ngrams[ngram] += 1

    return dict(ngrams)

# debug run
#count_ngrams(['привет привет как дела'], order=2, bos=True, eos=True)
count_ngrams(
    [
        'практическое сентября',
        'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
        'в офлайне в 32 12'
    ], 
    order=5
)
count_ngrams(['привет ' * 6], order=2, bos=False, eos=False)

{('привет',): 6, ('привет', 'привет'): 5}

In [4]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [5]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO

    ngram_count = counts[ngram] if ngram in counts else 0

    n = len(ngram)

    rest_count = 0
    voc_size = 0
    for ngram_text, count in counts.items():
        if V is None and len(ngram_text) == 1:
            voc_size += 1

        if len(ngram_text) == n and ngram_text[:-1] == ngram[:-1]:
            rest_count += count

    if V is None:
        V = voc_size

    prob = (ngram_count + k) / (rest_count + (k * V))

    return prob

# debug run
counts = count_ngrams(
    [
        'практическое сентября',
        'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
        'в офлайне в 32 12'
    ],
    order=4
)
#pprint(counts)

print(calculate_ngram_prob(('в', 'офлайне'), counts), 0.5)
print(calculate_ngram_prob(('в', ), counts), 4/12)
print(calculate_ngram_prob(('в', ), counts, k=0.5), 0.25)
print(calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts), 1.0)
print(calculate_ngram_prob(('в', 'офлайне'), counts, k=1), 0.1875)
print(calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5), 0.25)
print(calculate_ngram_prob(('в', 'онлайне'), counts, k=0), 0.0)
print(calculate_ngram_prob(('в', 'онлайне'), counts, k=1), 0.0625)
print(calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5), 0.25)

0.5 0.5
0.16 0.3333333333333333
0.14516129032258066 0.25
1.0 1.0
0.1875 0.1875
0.25 0.25
0.0 0.0
0.0625 0.0625
0.25 0.25


In [6]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [7]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)

    def _tokenize(self, text:str) -> list[str]:
        return re.split(r"\s+", text.strip())

    def _count_ngrams(self, train_text: List[str]) -> Dict[Tuple[str], int]:
        ngrams = defaultdict(int)

        for text in train_text:
            tokens = self._tokenize(text)

            if self.bos:
                tokens = ["<s>"] + tokens

            if self.eos:
                tokens = tokens + ["</s>"]

            for n in range(0, self.order):
                for pos in range(0, len(tokens)):
                    ngram = tuple(tokens[pos:pos+n+1])

                    if len(ngram) < n + 1:
                        continue

                    if ngram not in ngrams:
                        ngrams[ngram] = 0

                    ngrams[ngram] += 1

        return dict(ngrams)

    def _calculate_ngram_proba(self, ngram: Tuple[str]) -> float:

        ngram_count = self.ngrams_count[ngram] if ngram in self.ngrams_count else 0

        n = len(ngram)

        rest_count = 0
        for ngram_text, count in self.ngrams_count.items():
            if len(ngram_text) == n and ngram_text[:-1] == ngram[:-1]:
                rest_count += count

        prob = (ngram_count + self.k) / (rest_count + (self.k * self.V))

        return prob

    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = self._count_ngrams(train_text)
        self.vocab = [ ngram for ngram in self.ngrams_count.keys() if len(ngram) == 1 ]

    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        proba = self._calculate_ngram_proba(ngram)
        return np.log(proba)

    def predict_log_proba(self, words: List[str], return_ngrams_count:bool=False) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки

        words = [ word for word in words if word.strip() != "" ]

        start = 1 - self.order
        end = 0
        ngrams_count = 0
        while end < len(words):
            actual_start = start if start > 0 else 0
            actual_end = end + 1

            ngram = tuple(words[actual_start:actual_end])

            logprob += self.predict_ngram_log_proba(ngram)

            start += 1
            end += 1

            ngrams_count += 1

        return logprob if not return_ngrams_count else (logprob, ngrams_count)

    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # 
        perplexity = 0
        n = 0

        for line in text:
            tokens = self._tokenize(line)
            log_proba, ngrams_count = self.predict_log_proba(tokens, return_ngrams_count=True)
            perplexity += log_proba
            n += ngrams_count

        perplexity = np.exp(-perplexity / n)

        return perplexity

# debur run
train_data = [
    "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
    "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
    "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"
]

lm = NgramLM(order=2)
lm.fit(train_data)

test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]

print(f"""
lm.V = {lm.V}

lm.predict_log_proba(['мы']) = {lm.predict_log_proba(['мы'])}
lm.predict_log_proba(["если"]) = {lm.predict_log_proba(["если"])}
lm.predict_log_proba(["по-моему"]) = {lm.predict_log_proba(["по-моему"])}

lm.ppl(['']) = {lm.ppl([''])} / {((3+1)/28 * 1/(3+14))**(-1/2)}
lm.ppl(['ЧТО']) = {lm.ppl(['ЧТО'])} / {((3+1)/28 * 1/(3+14) * 1/(14)) ** (-1/3)}
lm.ppl(test_data) = {lm.ppl(test_data)} / 6.99
""")


lm.V = 14

lm.predict_log_proba(['мы']) = -7.594318331665067
lm.predict_log_proba(["если"]) = -7.594318331665067
lm.predict_log_proba(["по-моему"]) = -6.901171151105122

lm.ppl(['']) = 15.288884851420656 / 10.908712114635714
lm.ppl(['ЧТО']) = 14.846584407951667 / 11.854729962672497
lm.ppl(test_data) = 7.334561964590594 / 6.99



In [8]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [9]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 

    probabilities = []

    for word in lm.vocab:
        ngram = tuple(prefix + list(word))
        proba = lm.predict_ngram_log_proba(ngram)

        probabilities.append({
            "ngram": ngram,
            "proba": proba,
        })

    probabilities.sort(key=lambda row: row["proba"], reverse=True)

    next_words = [ (row["ngram"][-1], row["proba"]) for row in probabilities[:topk] ]

    return next_words


predict_next_word(lm, ["по-моему"])

[('мы', -1.7346010553881064),
 ('</s>', -2.1400661634962708),
 ('<s>', -2.833213344056216),
 ('по-моему', -2.833213344056216)]

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [10]:
src_text = """
# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$


Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.

Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$

# 2. Предсказания с помощью языковой модели (4 балла)

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

"""

Предобработка текста

In [11]:
# приводим к нижнему регистру
normalized = src_text.strip().lower()

# удаляем строки с формулами
normalized = re.sub(r"^\s*\$\$.+\$\$\s*$", "", normalized, flags=re.MULTILINE)

# удаляем всё, кроме разделителей предложений, цифр и букв
normalized = re.sub(r"[^\w\d\.\?!\-\s]", " ", normalized)

# удаляем тире и лишние пробелы
normalized = re.sub(r"\s+-\s+", " ", normalized)
normalized = re.sub(r"\s+", " ", normalized)

# делим на предложения
sentences = re.split(r"[\.!?]+|\n+", normalized)

# удаляем лишние пробелы
sentences = [sentence.strip() for sentence in sentences if sentence.strip() != ""]


In [12]:
train_data = sentences

lm = NgramLM(order=2)
lm.fit(train_data)


print("по-моему")
pprint(predict_next_word(lm, ["по-моему"], topk=4))
print()

print("сэкономим")
pprint(predict_next_word(lm, ["сэкономим"], topk=4))
print()

print("нграм")
pprint(predict_next_word(lm, ["нграм"], topk=4))
print()

print("вероятности")
pprint(predict_next_word(lm, ["вероятности"], topk=4))
print()


по-моему
[('мы', -4.382026634673881),
 ('сэкономим', -4.787491742782046),
 ('<s>', -5.480638923341991),
 ('языковые', -5.480638923341991)]

сэкономим
[('уйму', -4.386184644822546),
 ('нграм', -4.791649752930709),
 ('сэкономим', -4.791649752930709),
 ('<s>', -5.484796933490655)]

нграм
[('и', -4.402645921876617),
 ('в', -4.808111029984782),
 ('</s>', -4.808111029984782),
 ('языковой', -4.808111029984782)]

вероятности
[('</s>', -4.787491742782046),
 ('для', -4.787491742782046),
 ('хорошо', -4.787491742782046),
 ('<s>', -5.480638923341991)]



Попробуем сгенерировать короткий текст

In [13]:
first_word = "для"
length = 10

text = first_word
next_word = first_word

for _ in range(0, length):
    next_word = predict_next_word(lm, [next_word], topk=1)[0][0]
    text += " " + next_word

text

'для часто встречающихся нграм и лексически корректные тексты </s> <s> языковые'

In [14]:
first_word = "что"
length = 10

text = first_word
next_word = first_word

for _ in range(0, length):
    next_word = predict_next_word(lm, [next_word], topk=1)[0][0]
    text += " " + next_word

text

'что распределение вероятностей через количество нграм и лексически корректные тексты </s>'

In [15]:
first_word = "по-моему"
length = 10

text = first_word
next_word = first_word

for _ in range(0, length):
    next_word = predict_next_word(lm, [next_word], topk=1)[0][0]
    text += " " + next_word

text

'по-моему мы сэкономим уйму времени если в обучающей выборке </s> <s>'